In [1]:
!pip install transformers

     |████████████████████████████████| 1.9MB 6.8MB/s 
     |████████████████████████████████| 3.2MB 35.6MB/s 
     |████████████████████████████████| 890kB 37.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=71efd89b8b951ae10aaebf4a66d222ba0a8d8f4d7e3f9f94d435cf3040e13ba6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
import argparse
import cv2
import numpy as np
import torchvision
from torch.autograd import Function
from torchvision import models, transforms
import sys
import torch.nn as nn
from torch.optim import Adam
from transformers import GPT2Model, GPT2Config, GPT2ForSequenceClassification, GPT2Tokenizer

In [3]:
path_to_model = '/content/drive/MyDrive/gpt-resnet.torch'

example1 = '/content/example1.png'
example2 = '/content/example2.png'
example3 = '/content/example3.png'

meme1 = '/content/01235.png'
meme2 = '/content/01498.png'
meme3 = '/07486.png'

In [4]:
class concat_model(nn.Module):
    def __init__(self):
        super(concat_model, self).__init__()
        
        self.gpt2_model = GPT2ForSequenceClassification.from_pretrained('gpt2-large')
        num_ftrs = self.gpt2_model.score.in_features
        self.gpt2_model.score = nn.Linear(num_ftrs, 2_000)
        
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        
        self.vision_model = torchvision.models.resnet152(pretrained=True)
        num_ftrs = self.vision_model.fc.in_features
        self.vision_model.fc = nn.Linear(num_ftrs, 2000)
        input_size = 224
        self.vision_model = self.vision_model
        
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax()

        self.linear1 = nn.Linear(4000, 2000)
        self.linear2 = nn.Linear(2000, 2000)
        self.linear3 = nn.Linear(2000, 2000)
        self.linear4 = nn.Linear(2000, 2)
        
    def forward(self, data):
        images = data[0]
        gpt2_encoded = data[1]
            
        vision_out = self.vision_model(images)
        
        
        all_out = []
        for w in range(len(gpt2_encoded)):
            gpt2_out = self.gpt2_model(gpt2_encoded[w])[0]
            all_out.append(gpt2_out[0])
        gpt2_out = torch.stack(all_out)
        
        cat = torch.cat((vision_out,gpt2_out),dim=1)

        out1 = self.linear1(cat)
        out1 = self.relu(out1)
        out1 = self.linear2(out1)
        out1 = self.relu(out1)
        out1 = self.linear3(out1)
        out1 = self.relu(out1)
        out1 = self.linear4(out1)
        
        return out1

In [5]:
class FeatureExtractor():
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        for name, module in self.model._modules.items():
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x

In [6]:
class ModelOutputs():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """

    def __init__(self, model, feature_module, target_layers):
        self.model = model
        self.feature_module = feature_module
        self.feature_extractor = FeatureExtractor(self.feature_module, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations = []
        for name, module in self.model._modules.items():
            if module == self.feature_module:
                target_activations, x = self.feature_extractor(x)
            elif "avgpool" in name.lower():
                x = module(x)
                x = x.view(x.size(0),-1)
            else:
                x = module(x)

        return target_activations, x

In [7]:
def preprocess_image(img):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    preprocessing = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
    return preprocessing(img.copy()).unsqueeze(0)

In [8]:
def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

In [9]:
class GradCam:
    def __init__(self, model, feature_module, target_layer_names, use_cuda):
        self.model = model
        self.feature_module = feature_module
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        self.extractor = ModelOutputs(self.model, self.feature_module, target_layer_names)

    def forward(self, input_img):
        return self.model(input_img)

    def __call__(self, input_img, target_category=None):
        if self.cuda:
            input_img = input_img.cuda()

        features, output = self.extractor(input_img)

        if target_category == None:
            target_category = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][target_category] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = one_hot.cuda()
        
        one_hot = torch.sum(one_hot * output)

        self.feature_module.zero_grad()
        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

        target = features[-1]
        target = target.cpu().data.numpy()[0, :]

        weights = np.mean(grads_val, axis=(2, 3))[0, :]
        cam = np.zeros(target.shape[1:], dtype=np.float32)

        for i, w in enumerate(weights):
            cam += w * target[i, :, :]

        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, input_img.shape[2:])
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)
        return cam

In [10]:
class GuidedBackpropReLU(Function):
    @staticmethod
    def forward(self, input_img):
        positive_mask = (input_img > 0).type_as(input_img)
        output = torch.addcmul(torch.zeros(input_img.size()).type_as(input_img), input_img, positive_mask)
        self.save_for_backward(input_img, output)
        return output

    @staticmethod
    def backward(self, grad_output):
        input_img, output = self.saved_tensors
        grad_input = None

        positive_mask_1 = (input_img > 0).type_as(grad_output)
        positive_mask_2 = (grad_output > 0).type_as(grad_output)
        grad_input = torch.addcmul(torch.zeros(input_img.size()).type_as(input_img),
                                   torch.addcmul(torch.zeros(input_img.size()).type_as(input_img), grad_output,
                                                 positive_mask_1), positive_mask_2)
        return grad_input


In [11]:
class GuidedBackpropReLUModel:
    def __init__(self, model, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        def recursive_relu_apply(module_top):
            for idx, module in module_top._modules.items():
                recursive_relu_apply(module)
                if module.__class__.__name__ == 'ReLU':
                    module_top._modules[idx] = GuidedBackpropReLU.apply

        # replace ReLU with GuidedBackpropReLU
        recursive_relu_apply(self.model)

    def forward(self, input_img):
        return self.model(input_img)

    def __call__(self, input_img, target_category=None):
        if self.cuda:
            input_img = input_img.cuda()

        input_img = input_img.requires_grad_(True)

        output = self.forward(input_img)

        if target_category == None:
            target_category = np.argmax(output.cpu().data.numpy())

        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][target_category] = 1
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)
        if self.cuda:
            one_hot = one_hot.cuda()

        one_hot = torch.sum(one_hot * output)
        one_hot.backward(retain_graph=True)

        output = input_img.grad.cpu().data.numpy()
        output = output[0, :, :, :]

        return output

In [12]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--use-cuda', action='store_true', default=False,
                        help='Use NVIDIA GPU acceleration')
    parser.add_argument('--image-path', type=str, default='./examples/both.png',
                        help='Input image path')
    args = parser.parse_args()
    args.use_cuda = args.use_cuda and torch.cuda.is_available()
    if args.use_cuda:
        print("Using GPU for acceleration")
    else:
        print("Using CPU for computation")

    return args

In [13]:
def deprocess_image(img):
    """ see https://github.com/jacobgil/keras-grad-cam/blob/master/grad-cam.py#L65 """
    img = img - np.mean(img)
    img = img / (np.std(img) + 1e-5)
    img = img * 0.1
    img = img + 0.5
    img = np.clip(img, 0, 1)
    return np.uint8(img*255)

In [40]:
model = concat_model()
model.load_state_dict(torch.load(path_to_model, map_location=torch.device('cpu')))

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


<All keys matched successfully>

In [ ]:
model = model.vision_model
grad_cam = GradCam(model=model, feature_module=model.layer4, \
                       target_layer_names=["2"], use_cuda=False)

In [ ]:
img = cv2.imread(meme3, 1)
img = np.float32(img) / 255
# Opencv loads as BGR:
img = img[:, :, ::-1]
input_img = preprocess_image(img)

In [ ]:
# If None, returns the map for the highest scoring category.
# Otherwise, targets the requested category.
target_category = None
grayscale_cam = grad_cam(input_img, target_category)

grayscale_cam = cv2.resize(grayscale_cam, (img.shape[1], img.shape[0]))
cam = show_cam_on_image(img, grayscale_cam)

gb_model = GuidedBackpropReLUModel(model=model, use_cuda=False)
gb = gb_model(input_img, target_category=target_category)
gb = gb.transpose((1, 2, 0))

cam_mask = cv2.merge([grayscale_cam, grayscale_cam, grayscale_cam])
cam_gb = deprocess_image(cam_mask*gb)
gb = deprocess_image(gb)

cv2.imwrite('/content/cam.jpg', cam)
cv2.imwrite('/content/gb.jpg', gb)
cv2.imwrite('/content/cam_gb.jpg', cam_gb)

True

In [16]:
import pandas as pd

example_data = [['example1', '/content/example1.png', 1, 'love the way you smell today'],
                ['example2', '/content/example2.png', 1, 'your wrinkle cream is working great'],
                ['example3', '/content/example3.png', 1, 'look at how many people love you'],
                ['01235', '/content/01235.png', 1, 'when you\'re feeling horny asf but your habibi ...\n'],
                ['01498', '/content/01498.png', 0, 'democrats claim daca kids shouldn\'t have to pa...\n'],
                ['07486', '/content/07486.png', 1, 'what\'s the difference between a walrus and a l...\n']]

example_frame = pd.DataFrame(example_data, columns = ['id', 'img', 'label', 'text']) 

In [17]:
example_frame

,id,img,label,text
0,example1,/content/example1.png,1,love the way you smell today
1,example2,/content/example2.png,1,your wrinkle cream is working great
2,example3,/content/example3.png,1,look at how many people love you
3,01235,/content/01235.png,1,when you're feeling horny asf but your habibi ...
4,01498,/content/01498.png,0,democrats claim daca kids shouldn't have to pa...
5,07486,/content/07486.png,1,what's the difference between a walrus and a l...


In [26]:
import os
from PIL import Image

def get_example_images():
    image_ids = range(example_frame.values.shape[0])
    
    images = [
        Image.open(
            os.path.join(example_frame.loc[i, "img"])
        ).convert("RGB")
        for i in image_ids
    ]
    
    labels = [
        example_frame.loc[i, "label"]
        for i in image_ids
    ]
    
    text = [
        example_frame.loc[i, "text"]
        for i in image_ids
    ]
    
    image_transform = torchvision.transforms.Compose(
        [
            torchvision.transforms.Resize(size=(224, 224)),
            torchvision.transforms.ToTensor()
        ]
    )
    
    tensor_img = torch.stack(
        [image_transform(image) for image in images]
    )
    
    labels = torch.from_numpy(np.array(labels))
    
    return tensor_img, labels, text

In [29]:
tensor_img, labels, text = get_example_images()

In [36]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def get_enc_gpt(text):
    all_out = []
    for w in range(len(text)):
        inputs = torch.tensor([tokenizer.encode(text[w])])
        all_out.append(inputs)
    return all_out

In [41]:
from tqdm import tqdm_notebook as tqdm_n
import tqdm

softmax = torch.nn.Softmax(dim=1)

preds = []
glabels = []

gpt_encoded = get_enc_gpt(text)
predictions = softmax(model([tensor_img,gpt_encoded]))
arg_preds = torch.argmax(predictions, dim=1)
    
preds.append(arg_preds)

In [42]:
preds

[tensor([0, 0, 0, 1, 0, 0])]

In [43]:
labels

tensor([1, 1, 1, 1, 0, 1])